In [1]:
import requests
import time
import random
import sqlite3
from bs4 import BeautifulSoup
import re
import random


In [2]:
conn = sqlite3.connect('exposed.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS forum_urls
            (url TEXT)''')

conn.commit()

In [3]:
user_agents = [ 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36', 
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36', 
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36', 
    'Mozilla/5.0 (iPhone; CPU iPhone OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148', 
    'Mozilla/5.0 (Linux; Android 11; SM-G960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.72 Mobile Safari/537.36' 
]

In [4]:
def create_tables(c):
    cursor = c.cursor()
    # Create a table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS topics
                (url TEXT,title text,author text,replies integer,views integer,last_post text)''')
    c.commit()
    cursor.execute('''CREATE TABLE IF NOT EXISTS posts
                (url TEXT,thread_title text,author text,post_text text,post_order integer,user_id text)''')
    c.commit()
create_tables(conn)

In [5]:
def scrape_forums(url):
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    all_links = soup.find_all("a")
    pattern = re.compile(r".*Forum-.*")
    # Filter and print the links that match the pattern
    for link in all_links:
        href = link.get("href")
        if href and pattern.match(href):
            print(href)
            cursor.execute('''INSERT INTO forum_urls (url) VALUES (?)''', (href,))
            conn.commit()

In [6]:
def scrape_topics(soup):
    topics = soup.find_all("tr", class_="inline_row")
    for topic in topics:
        try:
            elements = topic.find_all(class_="trow2")
            if elements is None:
              elements = topic.find_all(class_="trow1") 
               
            topic_url = elements[1].find("a").get("href")
            #print("topic: ",topic_url)
            title = elements[1].find("a").text.strip()
            #print("title: ",title)
            author = elements[1].find("div", class_="author").text.strip()
            #print("author: ",author)
            #replied = elements[2].text.strip()
            #print("replies: ",replied)
            cursor.execute("INSERT or ignore INTO topics VALUES (?,?,?,?,?,?,?)",(topic_url,title, author, None,None,None,None))
            conn.commit()
            print(f"Inserted {topic_url}")
        except Exception as e:
            print(e)
            elements = topic.find_all(class_="trow1")
            topic_url = elements[1].find("a").get("href")
            continue


In [7]:
def scrape_forum_pages(forum_url):
    url = f"https://exposed.vc/{forum_url[0]}"
    print(f"scraping {url}")
    user_agent = random.choice(user_agents) 
    headers = {'User-Agent': user_agent}
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    scrape_topics(soup)

#Because I have already scraped all pages I will only scrape the fist page, as there no need for dublicate threads
"""
    try:
        page_count = int(soup.find(class_="pages").text.strip().split("(")[-1].replace("):", ""))
    except Exception as e:
        page_count = None
        
    # Filter and print the links that match the pattern
    if page_count != None:
        
        for i in range(2,page_count):
            print(f"scraping page {i}")
            page = requests.get(url+f"?page={i}",headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            scrape_topics(soup)
             """

'\n    try:\n        page_count = int(soup.find(class_="pages").text.strip().split("(")[-1].replace("):", ""))\n    except Exception as e:\n        page_count = None\n        \n    # Filter and print the links that match the pattern\n    if page_count != None:\n        \n        for i in range(2,page_count):\n            print(f"scraping page {i}")\n            page = requests.get(url+f"?page={i}",headers=headers)\n            soup = BeautifulSoup(page.content, \'html.parser\')\n            scrape_topics(soup)\n             '

In [9]:
cursor.execute("select url from forum_urls ")
rows = cursor.fetchall()
for row in rows:
    scrape_forum_pages(row)
    time.sleep(1)

scraping https://exposed.vc/Forum-Databases


ConnectionError: HTTPSConnectionPool(host='exposed.vc', port=443): Max retries exceeded with url: /Forum-Databases (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000024AA309C040>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))

In [7]:
# url, title, pk
# It only scrape the fist page as I find it the most valuable
def scrape_posts(topic):    
    url = f"https://exposed.vc/{topic[0]}"
    print(f"scraping {url}")
    user_agent = random.choice(user_agents)
    headers = {'User-Agent': user_agent}
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    #get_topic_timestamp(soup,topic[-1])
    posts = soup.find_all("div", class_="post") 
    for post in posts:
        try:
            username = post.find("div", class_="post__user-profile").text.strip()
            body_text = post.find("div", class_="body").text.strip()
            order = int(post.find(class_="postbit-number").text.strip().replace("#", ""))
            user_id = post.find("div", class_="post__user-profile").find("a").get("href").split("/")[-1]
            timestamp = soup.find("div" ,class_="showthread-desc").text.strip().split(" - ")[-1]
            cursor.execute("INSERT or ignore INTO posts VALUES (?,?,?,?,?,?)",(topic[0],topic[1],username,body_text,order,timestamp))
            conn.commit()
            
        except Exception as e:
            print(e)
            continue
    print(f"Inserted {url}")
def get_topic_timestamp(soup,topic_pk):
    timestamp = soup.find("div" ,class_="showthread-desc").text.strip().split(" - ")[-1]
    #update timestamp of topic using the pk
    cursor.execute("UPDATE topics SET last_post = ? WHERE pk = ?",(timestamp,topic_pk))
    conn.commit()


In [14]:
#scrape_posts(["Thread-2022-NVIDIA-Employee-database","test"])

cursor.execute("select topics.url,topics.title from topics left join posts on topics.url = posts.url where posts.url is NULL ")
rows = cursor.fetchall()
for row in rows:
    scrape_posts(row)